In [12]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

#defined -ve as clockwise and +ve as anti-clockwise rotation

In [13]:
"""initial parameters"""

QDs = 24 #number of quantum dots
voltage = 1 #applied voltage to capacitors in inner disck ("permanent charges"/charged segments)
capacitance_density = 0.394 #F/m^2, of inner segment capacitors
Qpos = 2 * 1.6e-19 #charge of positively charged quantum dot
Qneg = 2 *1.6e-19 #absolute value of charge of negative quantum dot
r0 = 110e-9 #distance between outer edge of inner disc and center of quantum dot directly above 
thickness = 30e-9 #approximate thickeness of inner disc
density = 3e3 #kg/m^3, desity of inner disc material (approxmiate)
epsilon = 8.854e-12 #currently set to epsilon 0
Stoppage_time = 30e-9 #double quoted 15ns, time spent stopped between steps, time needed to discharge and charge quantum dot
increment_step = math.pi/180 #one degree, recaclulate forces etc after motor has rotated by this step

In [14]:
"""functions"""

def intertia( density, thickness, R, r0):
	"""Calculates inertia of inner rotating disk"""
	I = 0.5 * density * math.pi * thickness * ( ( R-r0 ) ** 4 )
	return I 

def permanent_charge_magnitude( R, r0, QDs, capacitance_density, voltage):
    """Calculates magnitude of total charge stored in inner segment"""
    segment_area = ( 3 * math.pi * ( ( R-r0 ) ** 2 ) )/ (2 * QDs) #see notebook for deriviation
    capacitance = capacitance_density * segment_area #capacitance of one segment
    stored_charge = capacitance * voltage #stored charge in one segment
    return stored_charge

def permanent_charges( Qfixed_pos, Qfixed_neg, QDs):
	"""List of permanent charges of inner segments"""
	Q = [] 
	charge = 0
    #half as many inner segments as quantum dots
	for m in range( 0, int( QDs / 2 ) ): #'top' segment +ve (Q[0]), and all even numbered segments
		if ( m % 2 == 0):
			charge = Qfixed_pos
		else:
			charge = Qfixed_neg #odd numbered segments = -ve
		Q.append( (-1) * charge )
	return Q

def distances( R, r0, QDs, phi):
	"""Works out distances between 'permanent' segment charge and all quantum dots"""
    #inner segments are not point charges! Need to adjust this to be more realistic
	dist =[]
	for i in range(0, QDs):
		dist.append( math.sqrt( 2 * ((R **2) - R * r0) * (1 - math.cos( i * phi )) + (r0 **2) ) ) #see notebook for deriviation
	return dist

def perpendicular_component_fraction( R, Phi, r, r0):
	"""List of sins needed to correct force into tangential component to calculate torque"""
	sins = []
	for n in range( 0, int(QDs/2) +1 ): #first half of quantum dots
		if (((2 * R * math.sin( (n * Phi)/2 ) )**2) - (r[n]**2) - (r0**2) )/( -2 * r[n] * r0 ) > 1 or (((2 * R * math.sin( (n * Phi)/2 ) )**2) - (r[n]**2) - (r0**2) )/( -2 * r[n] * r0 ) < -1 :
			sins.append( 0 ) #corrects for computer error
		else:
			sins.append( math.sin( math.acos( (((2 * R * math.sin( (n * Phi)/2 ) )**2) - (r[n]**2) - (r0**2) )/( -2 * r[n] * r0 ) ) ) )
	for n in range( int(QDs/2) +1, QDs): #second half of qunatum dots, sin is -ve to keep with convention of -ve = clockwise
		sins.append( -1 * math.sin( math.acos( (((2 * R * math.sin( (n * Phi)/2 ) )**2) - (r[n]**2) - (r0**2) )/( -2 * r[n] * r0 ) ) ) )
	return sins

def temp_charges( m, Qpos, Qneg, QDs): 
	"""Charges of QDs, the order is with respect to permanent charge Q[m], with m starting from 0"""
	q = []
	charge = 0
	for n in range( 0, QDs, 4):
		q.append( 0 ) #dot 'directly above' each segment = 0 
		if (m % 2 == 0): #next dot along is charged opposite to current segment of interest
			q.append( -1 * Qneg ) #-vely charged QD
		else:
			q.append( Qpos ) #-vely charge QD

		q.append( 0 ) #next dot is 0 as above next segment
		if ( q[n+1] < 0): #next dot is charged oppositely to the previous QD
			q.append( Qpos )
		else:
			q.append( (-1 * Qneg) )
	return q

def Coulomb_force( Qpos, Qneg, r, Q, QDs, epsilon, sins):
	"""The summed coulomb torque on inner circle.  -ve is defined as turning clockwise, +ve as anticlockwise"""
	torque = 0
	q= temp_charges( 0, Qpos, Qneg, QDs) #charges QDs
	for n in range( 0, QDs):
		torque += q[n] * sins[n] / r[n] #adds up from each QD w.r.t. one segment
	torque =  (R-r0) * (QDs/2) * ( Q[0] / (4 * math.pi * epsilon) ) * torque #total torque on entire inner disc 
	return torque

def time_per_increment( phi, Inertia, torque):
    """Time taken to turn angle phi"""
    tau = 0
    tau = math.sqrt( ( 2 * phi * Inertia ) / abs(torque) ) #from rotational equivalent of SUVAT
    return tau

In [15]:
phi = (2 * math.pi ) / QDs #angle between each quantum dot
R = (70e-9)/( math.sin( phi/2 ) ) #inner disc radius, 70e-9 comes from distance between each QD needed to prevent tunneling
Inertia = intertia( density, thickness, R, r0) #intertia of inner disc
Qfixed_pos = permanent_charge_magnitude( R, r0, QDs, capacitance_density, voltage ) #charge of positive segment
Qfixed_neg = Qfixed_pos #absolute value of charge of negative segmentr = distances( R, r0, QDs, phi)

In [16]:
r = distances( R, r0, QDs, phi)
Q = permanent_charges( Qfixed_pos, Qfixed_neg, QDs)
sins = perpendicular_component_fraction( R, phi, r, r0)
torque = Coulomb_force( Qpos, Qneg, r, Q, QDs, epsilon, sins)
tau = time_per_increment( phi, Inertia, torque)
period = tau * QDs

In [17]:
print ("Torque = ", torque)
print ( "Time per step =" ,tau )
print ( "Period = " , period)

Radius =  [1.1e-07, 1.6637245348801708e-07, 2.708457789098976e-07, 3.82125728208298e-07, 4.906280294878659e-07, 5.924453120408606e-07, 6.850778982263932e-07, 7.665976816540682e-07, 8.354325765478855e-07, 8.903052179549791e-07, 9.302182632711936e-07, 9.544546400939975e-07, 9.625816605756547e-07, 9.544546400939975e-07, 9.302182632711937e-07, 8.903052179549792e-07, 8.354325765478857e-07, 7.665976816540685e-07, 6.850778982263934e-07, 5.924453120408608e-07, 4.906280294878662e-07, 3.8212572820829817e-07, 2.708457789098979e-07, 1.6637245348801738e-07]
Torque =  1.380842011850625e-21
Time per step = 4.20746478903537e-05
Period =  0.001009791549368489
